In [5]:
import sys
sys.path.append(r'C:\Users\lwang\Documents\GitHub\DataAnalysis')

In [6]:
import numpy as np
import sys, os
from pathlib import Path
import pandas as pd
import copy
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
from plotly.colors import n_colors
import math
import datetime as dt
from sklearn.metrics import r2_score
import plotly.colors
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu
from scipy.interpolate import griddata
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
import scipy
import importlib
import pickle
import psutil 
import gc

# ----- Internal Dependencies -------#
nb_dir = os.path.split(os.getcwd())[0]
sys.path.append("../..")
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from General import FileRead
from General import AccuracyMetrics
from General.GeneralFunctions import get_add_to_dict
from General import TableManipulations
from FeatureGeneration import GeneralFeatures
from General import FileWrite

from Plot.PdfHelper import PdfHelper
from Plot.PlotMaker import PlotMaker
# from Plot import Voltage_Plots
from Plot import PlotFunctions
# from Plot import AdHocPlots
# from Plot import Sim_Plots

# widget and notebook stuff
from ipywidgets import interact, fixed
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"

load data

In [47]:
# training_group = ['eBlinqMayStudyCtrl', 'eBlinqMayStudyTest', 'eBlinqJuneStudyCtrl',
#        'eBlinqJuneStudyTest', 'Pre-Piv-2-training',
#        'Pre-Piv-2-validation', 'eBlinq19c', 'eBlinq20', 'eBlinq22',
#        'eBlinq23', 'eBlinq25', 'eBlinqRingOverlay&NiMNA', 'iBlinqAcet',
#        'eBlinq14', 'eBlinq15', 'eBlinq16', 'eBlinq17', 'eBlinq18',
#        'eBlinq19a', 'eBlinq19b', 'iBlinq16', 'iBlinqAdhesive',
#        'iBlinqFW231', 'iBlinqOverlay']
training_group = ['eBlinq19c', 'eBlinq20', 'eBlinq22','eBlinq23','eBlinq25','iBlinqAcet','Pre-Piv-2-training','eBlinqMayStudyCtrl','Pre-Piv-2-validation','eBlinqJuneStudyCtrl']
training_time_window = [0.083,5]
training_tier = ['Tier 1: 3/3 Clinical Use', 'Tier 1w-a: 2/3 CLN, 3/3 Wide', 'Tier 1w-b: 1/3 CLN, 3/3 Wide', 'Tier 1w-c: 0/3 CLN, 3/3 Wide']
training_config = ['Control', 'Unspecified']
training_chemistry = '2.6.11'
training_ivarm = ['No', 'Unspecified']

test_group = ['eBlinq19c', 'eBlinq20', 'eBlinq22','Pre-Piv-2-training','eBlinqMayStudyCtrl','Pre-Piv-2-validation','eBlinqJuneStudyCtrl']
test_tier = ['Tier 1: 3/3 Clinical Use', 'Tier 1w-a: 2/3 CLN, 3/3 Wide', 'Tier 1w-b: 1/3 CLN, 3/3 Wide', 'Tier 1w-c: 0/3 CLN, 3/3 Wide']
test_config = ['Control', 'Unspecified']
test_chemistry = '2.6.11'
test_ivarm = ['No', 'Unspecified']

all_training_channel = ['1','3','4']
iso_thresh = 30
feature_hour_short_window = '8hr'
feature_hour_long_window = '120hr'
feature_col_dict = {
    'feature_col_ch1': [
        'time_from_start_featureGeneration',
        'Temperature_alignment',
        'CE_alignment',
        'cur1',
        'deg1_103_featureGeneration',
        'mag1_103_featureGeneration',
        'deg1_1007_featureGeneration',
        'mag1_1007_featureGeneration',
        'deg1_10000_featureGeneration',
        'mag1_10000_featureGeneration',
        'ewm_average_8hr_deg1_10000_featureGeneration',
        'ewm_average_8hr_deg1_1007_featureGeneration',
        'ewm_average_8hr_deg1_103_featureGeneration',
        'ewm_average_8hr_mag1_10000_featureGeneration',
        'ewm_average_8hr_mag1_1007_featureGeneration',
        'ewm_average_8hr_mag1_103_featureGeneration',   
        f'ewm_average_{feature_hour_short_window}_iir_filt_model_output_s1_glucoseFeatureGeneration',
        f'estimate_standard_deviation_{feature_hour_short_window}_iir_filt_model_output_s1_glucoseFeatureGeneration',    
        f'ewm_average_{feature_hour_long_window}_iir_filt_model_output_s1_glucoseFeatureGeneration',
        f'estimate_standard_deviation_{feature_hour_long_window}_iir_filt_model_output_s1_glucoseFeatureGeneration',
    ],
    'feature_col_ch3': [
        'time_from_start_featureGeneration',
        'Temperature_alignment',
        'CE_alignment',
        'cur3',
        'deg3_103_featureGeneration',
        'mag3_103_featureGeneration',
        'deg3_1007_featureGeneration',
        'mag3_1007_featureGeneration',
        'deg3_10000_featureGeneration',
        'mag3_10000_featureGeneration',
        'ewm_average_8hr_deg3_10000_featureGeneration',
        'ewm_average_8hr_deg3_1007_featureGeneration',
        'ewm_average_8hr_deg3_103_featureGeneration',
        'ewm_average_8hr_mag3_10000_featureGeneration',
        'ewm_average_8hr_mag3_1007_featureGeneration',
        'ewm_average_8hr_mag3_103_featureGeneration',   
        f'ewm_average_{feature_hour_short_window}_iir_filt_model_output_s3_glucoseFeatureGeneration',
        f'estimate_standard_deviation_{feature_hour_short_window}_iir_filt_model_output_s3_glucoseFeatureGeneration',    
        f'ewm_average_{feature_hour_long_window}_iir_filt_model_output_s3_glucoseFeatureGeneration',
        f'estimate_standard_deviation_{feature_hour_long_window}_iir_filt_model_output_s3_glucoseFeatureGeneration',
    ],
    'feature_col_ch4': [
        'time_from_start_featureGeneration',
        'Temperature_alignment',
        'CE_alignment',
        'cur4',
        'deg4_103_featureGeneration',
        'mag4_103_featureGeneration',
        'deg4_1007_featureGeneration',
        'mag4_1007_featureGeneration',
        'deg4_10000_featureGeneration',
        'mag4_10000_featureGeneration',
        'ewm_average_8hr_deg4_10000_featureGeneration',
        'ewm_average_8hr_deg4_1007_featureGeneration',
        'ewm_average_8hr_deg4_103_featureGeneration',
        'ewm_average_8hr_mag4_10000_featureGeneration',
        'ewm_average_8hr_mag4_1007_featureGeneration',
        'ewm_average_8hr_mag4_103_featureGeneration',   
        f'ewm_average_{feature_hour_short_window}_iir_filt_model_output_s4_glucoseFeatureGeneration',
        f'estimate_standard_deviation_{feature_hour_short_window}_iir_filt_model_output_s4_glucoseFeatureGeneration',    
        f'ewm_average_{feature_hour_long_window}_iir_filt_model_output_s4_glucoseFeatureGeneration',
        f'estimate_standard_deviation_{feature_hour_long_window}_iir_filt_model_output_s4_glucoseFeatureGeneration',
    ],        
}
gluc_col = [
    'model_output_s1_calculateGlucose','model_output_s3_calculateGlucose','model_output_s4_calculateGlucose',
]
gluc_fault_col = ['model_output_is_valid_s1_channelFaultDetection','model_output_is_valid_s3_channelFaultDetection','model_output_is_valid_s4_channelFaultDetection']
info_ref = ['Study','Sensor_Id','Chemistry mfg','Sorting Tier mfg','IV Arm','Config']
retrofit_col = ['retro_model_output_s1_calculateGlucose','retro_model_output_s3_calculateGlucose','retro_model_output_s4_calculateGlucose']
ref_for_optim = 'cgm_adj'
ref_for_optim_list = [ref_for_optim] # 'ysi', 'acck'
ref_for_eval = ['cgm_adj','ysi', 'acck']
all_cols = set(feature_col_dict['feature_col_ch1'] + feature_col_dict['feature_col_ch3'] + feature_col_dict['feature_col_ch4'] + gluc_col + info_ref + retrofit_col + ref_for_optim_list + ref_for_eval + gluc_fault_col)

In [8]:
df_orig=pd.read_csv(r'S:\for Liang\data for optimization\agg_sample_df_ref_gModel_cgmadj_all2.csv')

c:\Users\lwang\Documents\GitHub\AlgoVenv1\lib\site-packages\IPython\core\interactiveshell.py:3269: DtypeWarning:

Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.



In [9]:
sensor_info_df = pd.read_excel(r'C:\Users\lwang\OneDrive - Biolinq Inc\Gen 1\Algorithm Development\neural network glucose model\all sensor info updated.xlsx')

In [10]:
sensor_info_df.columns

Index(['Study', 'sensor_id', 'sensor_id_corrected', 'Subject ID', 'IV Arm',
       'Self Apply', 'Location', 'Config', 'Sorting Tier mfg', 'Chemistry mfg',
       'Lot', 'Build', 'Wafer Lot #', 'Wafer Type', 'Wafer Batch - #',
       'Wafer Location', 'Cardinal', 'Quadrant', 'Edge Level', 'Outermost'],
      dtype='object')

In [11]:
df_orig = df_orig.merge(right=sensor_info_df,how='left',left_on='Sensor_Id',right_on='sensor_id',suffixes=(None,'2'))

In [12]:
df_orig['Config'].unique()

array(['Control', nan, 'Unspecified'], dtype=object)

In [13]:
df_orig['IV Arm'].unique()

array(['No', 'Yes', nan, 'Unspecified'], dtype=object)

In [14]:
df_orig['Study'].unique()

array(['eBlinqMayStudyCtrl', 'eBlinqMayStudyTest', 'eBlinqJuneStudyCtrl',
       'eBlinqJuneStudyTest', 'Pre-Piv-2-training',
       'Pre-Piv-2-validation', 'eBlinq19c', 'eBlinq20', 'eBlinq22',
       'eBlinq23', 'eBlinq25', 'eBlinqRingOverlay&NiMNA', 'iBlinqAcet',
       'eBlinq14', 'eBlinq15', 'eBlinq16', 'eBlinq17', 'eBlinq18',
       'eBlinq19a', 'eBlinq19b', 'iBlinq16', 'iBlinqAdhesive',
       'iBlinqFW231', 'iBlinqOverlay'], dtype=object)

In [15]:
df_orig['Chemistry mfg'].unique()

array(['2.6.11', nan, '2.5.11'], dtype=object)

In [27]:
df_orig['Chemistry mfg'].isna().sum()

15057221

In [16]:
df_orig['Sorting Tier mfg'].unique()

array(['Tier 1: 3/3 Clinical Use', nan, 'Tier 1w-a: 2/3 CLN, 3/3 Wide',
       'Tier 5: Non-Functional', 'Tier 2: 2/3 Pass CLN',
       'Tier 1w-b: 1/3 CLN, 3/3 Wide', 'Tier 1w-c: 0/3 CLN, 3/3 Wide'],
      dtype=object)

In [20]:
df_orig['IV Arm'].unique()

array(['No', 'Yes', nan, 'Unspecified'], dtype=object)

In [17]:
df = df_orig[all_cols].copy()

In [42]:
all_gModel = {}

In [48]:
x_train_all_ch = pd.DataFrame()
y_train_all_ch = np.empty((0,))
hypo_idx = df[ref_for_optim].le(100)
for ch_str in all_training_channel:
    print(f'Concatenate channel {ch_str}')
    df[f'ad_s{ch_str}'] = df[f'retro_model_output_s{ch_str}_calculateGlucose'].sub(df[ref_for_optim]).abs()
    df[f'ard_s{ch_str}'] = df[f'ad_s{ch_str}'].div(df[ref_for_optim]).mul(100)
    df[f'iso_s{ch_str}'] = df[f'ard_s{ch_str}']
    df.loc[hypo_idx,f'iso_s{ch_str}'] = df.loc[hypo_idx,f'ad_s{ch_str}']
    feature_col = feature_col_dict[f'feature_col_ch{ch_str}']
    col_for_train = feature_col + [f'model_output_s{ch_str}_calculateGlucose'] + ref_for_optim_list
    training_index = df['Study'].isin(training_group) & df['time_from_start_featureGeneration'].between(training_time_window[0],training_time_window[1]) & \
                            df['Chemistry mfg'].eq(training_chemistry) & df['Sorting Tier mfg'].isin(training_tier) &\
                            df['IV Arm'].isin(training_ivarm) & df['Config'].isin(training_config)
    len_training = training_index.sum()
    # valid_training_index = training_index & df[f'iso_s{ch_str}'].le(iso_thresh)
    valid_training_index = training_index & df[f'model_output_is_valid_s{ch_str}_channelFaultDetection']
    len_valid_training = valid_training_index.sum()
    df_training_valid = df.loc[valid_training_index,col_for_train].dropna().reset_index(drop=True)
    print(f'Training data has length {len_training}')
    print(f'Training data has length {len_valid_training}')
    good_data_ratio = len_valid_training/len_training
    print(f'Ratio of good data is {good_data_ratio}')
    x_train = df_training_valid[feature_col].copy()
    x_train.rename(columns={
        f'cur{ch_str}':'cur',
        f'deg{ch_str}_103_featureGeneration':'deg_103_featureGeneration',
        f'mag{ch_str}_103_featureGeneration':'mag_103_featureGeneration',
        f'deg{ch_str}_1007_featureGeneration':'deg_1007_featureGeneration',
        f'mag{ch_str}_1007_featureGeneration':'mag_1007_featureGeneration',
        f'deg{ch_str}_10000_featureGeneration':'deg_10000_featureGeneration',
        f'mag{ch_str}_10000_featureGeneration':'mag_10000_featureGeneration',
        f'ewm_average_8hr_deg{ch_str}_10000_featureGeneration':'ewm_average_8hr_deg_10000_featureGeneration',
        f'ewm_average_8hr_deg{ch_str}_1007_featureGeneration':'ewm_average_8hr_deg_1007_featureGeneration',
        f'ewm_average_8hr_deg{ch_str}_103_featureGeneration':'ewm_average_8hr_deg_103_featureGeneration',
        f'ewm_average_8hr_mag{ch_str}_10000_featureGeneration':'ewm_average_8hr_mag_10000_featureGeneration',
        f'ewm_average_8hr_mag{ch_str}_1007_featureGeneration':'ewm_average_8hr_mag_1007_featureGeneration',
        f'ewm_average_8hr_mag{ch_str}_103_featureGeneration':'ewm_average_8hr_mag_103_featureGeneration',
        f'ewm_average_{feature_hour_short_window}_iir_filt_model_output_s{ch_str}_glucoseFeatureGeneration':f'ewm_average_{feature_hour_short_window}_iir_filt_model_output_glucoseFeatureGeneration',
        f'estimate_standard_deviation_{feature_hour_short_window}_iir_filt_model_output_s{ch_str}_glucoseFeatureGeneration':f'estimate_standard_deviation_{feature_hour_short_window}_iir_filt_model_output_glucoseFeatureGeneration',
        f'ewm_average_{feature_hour_long_window}_iir_filt_model_output_s{ch_str}_glucoseFeatureGeneration':f'ewm_average_{feature_hour_long_window}_iir_filt_model_output_glucoseFeatureGeneration',
        f'estimate_standard_deviation_{feature_hour_long_window}_iir_filt_model_output_s{ch_str}_glucoseFeatureGeneration':f'estimate_standard_deviation_{feature_hour_long_window}_iir_filt_model_output_glucoseFeatureGeneration',
    },inplace=True)
    x_train['Channel'] = ch_str
    x_train_all_ch = pd.concat([x_train_all_ch,x_train],ignore_index=True)
    y_train = df_training_valid[f'model_output_s{ch_str}_calculateGlucose'].sub(df_training_valid[ref_for_optim]).to_numpy()
    y_train_all_ch = np.concatenate((y_train_all_ch,y_train))

Concatenate channel 1
Training data has length 8076919
Training data has length 7042947
Ratio of good data is 0.871984354430198
Concatenate channel 3
Training data has length 8076919
Training data has length 7109035
Ratio of good data is 0.8801666823698492
Concatenate channel 4
Training data has length 8076919
Training data has length 7120045
Ratio of good data is 0.881529825915055


In [49]:
min_max_scaler = sklearn.preprocessing.MinMaxScaler()
x_train_all_ch_norm = min_max_scaler.fit_transform(x_train_all_ch)
err_model = MLPRegressor(hidden_layer_sizes=(10,7,5,1),activation='logistic',solver='adam',alpha=0.0005,max_iter=int(1e4),tol=1e-4,learning_rate='adaptive',random_state=20140806)
err_model.fit(x_train_all_ch_norm,y_train_all_ch)
all_gModel[f'min_max_scaler'] = min_max_scaler
all_gModel[f'err_model'] = err_model

In [ ]:
for ch_str in all_training_channel:
    print(f'Evaluating channel {ch_str}')
    feature_col = feature_col_dict[f'feature_col_ch{ch_str}']
    col_for_train = feature_col + [f'model_output_s{ch_str}_calculateGlucose'] + ref_for_optim_list
    training_index = df['Study'].isin(training_group) & df['time_from_start_featureGeneration'].between(training_time_window[0],training_time_window[1]) & \
                            df['Chemistry mfg'].eq(training_chemistry) & df['Sorting Tier mfg'].isin(training_tier) &\
                            df['IV Arm'].isin(training_ivarm) & df['Config'].isin(training_config)
    # valid_training_index = training_index & df[f'iso_s{ch_str}'].le(iso_thresh)
    valid_training_index = training_index & df[f'model_output_is_valid_s{ch_str}_channelFaultDetection']
    df_training_valid = df.loc[valid_training_index,:].dropna(subset=col_for_train).reset_index(drop=True)
    x_train = df_training_valid[feature_col].copy()
    x_train['Channel'] = ch_str
    x_train.rename(columns={
        f'cur{ch_str}':'cur',
        f'deg{ch_str}_103_featureGeneration':'deg_103_featureGeneration',
        f'mag{ch_str}_103_featureGeneration':'mag_103_featureGeneration',
        f'deg{ch_str}_1007_featureGeneration':'deg_1007_featureGeneration',
        f'mag{ch_str}_1007_featureGeneration':'mag_1007_featureGeneration',
        f'deg{ch_str}_10000_featureGeneration':'deg_10000_featureGeneration',
        f'mag{ch_str}_10000_featureGeneration':'mag_10000_featureGeneration',
        f'ewm_average_8hr_deg{ch_str}_10000_featureGeneration':'ewm_average_8hr_deg_10000_featureGeneration',
        f'ewm_average_8hr_deg{ch_str}_1007_featureGeneration':'ewm_average_8hr_deg_1007_featureGeneration',
        f'ewm_average_8hr_deg{ch_str}_103_featureGeneration':'ewm_average_8hr_deg_103_featureGeneration',
        f'ewm_average_8hr_mag{ch_str}_10000_featureGeneration':'ewm_average_8hr_mag_10000_featureGeneration',
        f'ewm_average_8hr_mag{ch_str}_1007_featureGeneration':'ewm_average_8hr_mag_1007_featureGeneration',
        f'ewm_average_8hr_mag{ch_str}_103_featureGeneration':'ewm_average_8hr_mag_103_featureGeneration',
        f'ewm_average_{feature_hour_short_window}_iir_filt_model_output_s{ch_str}_glucoseFeatureGeneration':f'ewm_average_{feature_hour_short_window}_iir_filt_model_output_glucoseFeatureGeneration',
        f'estimate_standard_deviation_{feature_hour_short_window}_iir_filt_model_output_s{ch_str}_glucoseFeatureGeneration':f'estimate_standard_deviation_{feature_hour_short_window}_iir_filt_model_output_glucoseFeatureGeneration',
        f'ewm_average_{feature_hour_long_window}_iir_filt_model_output_s{ch_str}_glucoseFeatureGeneration':f'ewm_average_{feature_hour_long_window}_iir_filt_model_output_glucoseFeatureGeneration',
        f'estimate_standard_deviation_{feature_hour_long_window}_iir_filt_model_output_s{ch_str}_glucoseFeatureGeneration':f'estimate_standard_deviation_{feature_hour_long_window}_iir_filt_model_output_glucoseFeatureGeneration',
    },inplace=True)    
    y_train = df_training_valid[f'model_output_s{ch_str}_calculateGlucose'].sub(df_training_valid[ref_for_optim]).to_numpy()
    x_train_norm =  all_gModel[f'min_max_scaler'].transform(x_train)    
    predicted_err_train = all_gModel[f'err_model'].predict(x_train_norm)
    df_training_valid[f'model_output_s{ch_str}_correction'] = df_training_valid[f'model_output_s{ch_str}_calculateGlucose'].sub(predicted_err_train)
    for ref_name in ref_for_eval:
        n_train_today = df_training_valid[f'model_output_s{ch_str}_calculateGlucose'].sub(df_training_valid[ref_name]).div(df_training_valid[ref_name]).count()
        mrd_train_today = df_training_valid[f'model_output_s{ch_str}_calculateGlucose'].sub(df_training_valid[ref_name]).div(df_training_valid[ref_name]).mean()
        stdrd_train_today = df_training_valid[f'model_output_s{ch_str}_calculateGlucose'].sub(df_training_valid[ref_name]).div(df_training_valid[ref_name]).std()
        mard_train_today = df_training_valid[f'model_output_s{ch_str}_calculateGlucose'].sub(df_training_valid[ref_name]).div(df_training_valid[ref_name]).abs().mean()
        n_train_correction = df_training_valid[f'model_output_s{ch_str}_correction'].sub(df_training_valid[ref_name]).div(df_training_valid[ref_name]).count()
        mrd_train_correction = df_training_valid[f'model_output_s{ch_str}_correction'].sub(df_training_valid[ref_name]).div(df_training_valid[ref_name]).mean()
        stdrd_train_correction = df_training_valid[f'model_output_s{ch_str}_correction'].sub(df_training_valid[ref_name]).div(df_training_valid[ref_name]).std()
        mard_train_correction = df_training_valid[f'model_output_s{ch_str}_correction'].sub(df_training_valid[ref_name]).div(df_training_valid[ref_name]).abs().mean()
        n_train_retro = df_training_valid[f'retro_model_output_s{ch_str}_calculateGlucose'].sub(df_training_valid[ref_name]).div(df_training_valid[ref_name]).count()
        mrd_train_retro = df_training_valid[f'retro_model_output_s{ch_str}_calculateGlucose'].sub(df_training_valid[ref_name]).div(df_training_valid[ref_name]).mean()
        stdrd_train_retro = df_training_valid[f'retro_model_output_s{ch_str}_calculateGlucose'].sub(df_training_valid[ref_name]).div(df_training_valid[ref_name]).std()
        mard_train_retro = df_training_valid[f'retro_model_output_s{ch_str}_calculateGlucose'].sub(df_training_valid[ref_name]).div(df_training_valid[ref_name]).abs().mean()
        print(f'Training data for channel {ch_str} glucose by model today, vs {ref_name}, N = {n_train_today}, MRD = {mrd_train_today}, StdRD = {stdrd_train_today}, MARD = {mard_train_today}')
        print(f'Training data for channel {ch_str} glucose by nn_model, vs {ref_name}, N = {n_train_correction}, MRD = {mrd_train_correction}, StdRD = {stdrd_train_correction}, MARD = {mard_train_correction}')
        print(f'Training data for channel {ch_str} glucose by retro, vs {ref_name}, N = {n_train_retro}, MRD = {mrd_train_retro}, StdRD = {stdrd_train_retro}, MARD = {mard_train_retro}')

    for test_data in test_group:
        test_index = df['Study'].eq(test_data) & df['time_from_start_featureGeneration'].between(training_time_window[0],training_time_window[1]) & \
                                df['Chemistry mfg'].eq(test_chemistry) & df['Sorting Tier mfg'].isin(test_tier) &\
                                df['IV Arm'].isin(test_ivarm) & df['Config'].isin(test_config)    
        # valid_test_index = test_index & df[f'iso_s{ch_str}'].le(iso_thresh)
        valid_test_index = test_index & df[f'model_output_is_valid_s{ch_str}_channelFaultDetection']
        df_test_valid = df.loc[valid_test_index,:].dropna(subset=col_for_train).reset_index(drop=True)
        print(f'Test data has shape {df_test_valid.shape}')
        x_test = df_test_valid[feature_col].copy()
        x_test['Channel'] = ch_str
        x_test.rename(columns={
            f'cur{ch_str}':'cur',
            f'deg{ch_str}_103_featureGeneration':'deg_103_featureGeneration',
            f'mag{ch_str}_103_featureGeneration':'mag_103_featureGeneration',
            f'deg{ch_str}_1007_featureGeneration':'deg_1007_featureGeneration',
            f'mag{ch_str}_1007_featureGeneration':'mag_1007_featureGeneration',
            f'deg{ch_str}_10000_featureGeneration':'deg_10000_featureGeneration',
            f'mag{ch_str}_10000_featureGeneration':'mag_10000_featureGeneration',
            f'ewm_average_8hr_deg{ch_str}_10000_featureGeneration':'ewm_average_8hr_deg_10000_featureGeneration',
            f'ewm_average_8hr_deg{ch_str}_1007_featureGeneration':'ewm_average_8hr_deg_1007_featureGeneration',
            f'ewm_average_8hr_deg{ch_str}_103_featureGeneration':'ewm_average_8hr_deg_103_featureGeneration',
            f'ewm_average_8hr_mag{ch_str}_10000_featureGeneration':'ewm_average_8hr_mag_10000_featureGeneration',
            f'ewm_average_8hr_mag{ch_str}_1007_featureGeneration':'ewm_average_8hr_mag_1007_featureGeneration',
            f'ewm_average_8hr_mag{ch_str}_103_featureGeneration':'ewm_average_8hr_mag_103_featureGeneration',
            f'ewm_average_{feature_hour_short_window}_iir_filt_model_output_s{ch_str}_glucoseFeatureGeneration':f'ewm_average_{feature_hour_short_window}_iir_filt_model_output_glucoseFeatureGeneration',
            f'estimate_standard_deviation_{feature_hour_short_window}_iir_filt_model_output_s{ch_str}_glucoseFeatureGeneration':f'estimate_standard_deviation_{feature_hour_short_window}_iir_filt_model_output_glucoseFeatureGeneration',
            f'ewm_average_{feature_hour_long_window}_iir_filt_model_output_s{ch_str}_glucoseFeatureGeneration':f'ewm_average_{feature_hour_long_window}_iir_filt_model_output_glucoseFeatureGeneration',
            f'estimate_standard_deviation_{feature_hour_long_window}_iir_filt_model_output_s{ch_str}_glucoseFeatureGeneration':f'estimate_standard_deviation_{feature_hour_long_window}_iir_filt_model_output_glucoseFeatureGeneration',
        },inplace=True)        
        x_test_norm = all_gModel[f'min_max_scaler'].transform(x_test)
        predicted_err_test = all_gModel[f'err_model'].predict(x_test_norm)
        df_test_valid[f'model_output_s{ch_str}_correction'] = df_test_valid[f'model_output_s{ch_str}_calculateGlucose'].sub(predicted_err_test)
        for ref_name in ref_for_eval:
            n_test_today = df_test_valid[f'model_output_s{ch_str}_calculateGlucose'].sub(df_test_valid[ref_name]).div(df_test_valid[ref_name]).count()
            mrd_test_today = df_test_valid[f'model_output_s{ch_str}_calculateGlucose'].sub(df_test_valid[ref_name]).div(df_test_valid[ref_name]).mean()
            stdrd_test_today = df_test_valid[f'model_output_s{ch_str}_calculateGlucose'].sub(df_test_valid[ref_name]).div(df_test_valid[ref_name]).std()
            mard_test_today = df_test_valid[f'model_output_s{ch_str}_calculateGlucose'].sub(df_test_valid[ref_name]).div(df_test_valid[ref_name]).abs().mean()
            n_test_correction = df_test_valid[f'model_output_s{ch_str}_correction'].sub(df_test_valid[ref_name]).div(df_test_valid[ref_name]).count()
            mrd_test_correction = df_test_valid[f'model_output_s{ch_str}_correction'].sub(df_test_valid[ref_name]).div(df_test_valid[ref_name]).mean()
            stdrd_test_correction = df_test_valid[f'model_output_s{ch_str}_correction'].sub(df_test_valid[ref_name]).div(df_test_valid[ref_name]).std()
            mard_test_correction = df_test_valid[f'model_output_s{ch_str}_correction'].sub(df_test_valid[ref_name]).div(df_test_valid[ref_name]).abs().mean()
            n_test_retro = df_test_valid[f'retro_model_output_s{ch_str}_calculateGlucose'].sub(df_test_valid[ref_name]).div(df_test_valid[ref_name]).count()
            mrd_test_retro = df_test_valid[f'retro_model_output_s{ch_str}_calculateGlucose'].sub(df_test_valid[ref_name]).div(df_test_valid[ref_name]).mean()
            stdrd_test_retro = df_test_valid[f'retro_model_output_s{ch_str}_calculateGlucose'].sub(df_test_valid[ref_name]).div(df_test_valid[ref_name]).std()
            mard_test_retro = df_test_valid[f'retro_model_output_s{ch_str}_calculateGlucose'].sub(df_test_valid[ref_name]).div(df_test_valid[ref_name]).abs().mean()
            print(f'Test data {test_data} for channel {ch_str} glucose by model today, vs {ref_name}, N = {n_test_today}, MRD = {mrd_test_today}, StdRD = {stdrd_test_today}, MARD = {mard_test_today}')
            print(f'Test data {test_data} for channel {ch_str} glucose by nn_model, vs {ref_name}, N = {n_test_correction}, MRD = {mrd_test_correction}, StdRD = {stdrd_test_correction}, MARD = {mard_test_correction}')
            print(f'Test data {test_data} for channel {ch_str} glucose by retro, vs {ref_name}, N = {n_test_retro}, MRD = {mrd_test_retro}, StdRD = {stdrd_test_retro}, MARD = {mard_test_retro}')

Evaluating channel 1
Training data for channel 1 glucose by model today, vs cgm_adj, N = 3371541, MRD = -0.01750548479317695, StdRD = 0.27337696286670365, MARD = 0.20150867412011042
Training data for channel 1 glucose by nn_model, vs cgm_adj, N = 3371541, MRD = 0.05298044365892625, StdRD = 0.23363571565428878, MARD = 0.1666428672439537
Training data for channel 1 glucose by retro, vs cgm_adj, N = 3371541, MRD = 0.02226348418674514, StdRD = 1.4062002545334604, MARD = 0.1883521213061804
Training data for channel 1 glucose by model today, vs ysi, N = 133436, MRD = -0.03213118375436274, StdRD = 0.23040955173830419, MARD = 0.1806567355975219
Training data for channel 1 glucose by nn_model, vs ysi, N = 133436, MRD = 0.07148766436056035, StdRD = 0.2183647050141432, MARD = 0.17547030750385423
Training data for channel 1 glucose by retro, vs ysi, N = 133436, MRD = 0.06364287759818127, StdRD = 0.18837668975708385, MARD = 0.14823901236725257
Training data for channel 1 glucose by model today, vs 

In [ ]:
# min_max_scaler.feature_names_in_ = np.array(['time_from_start',
#                     'ewm_average_8hr_iir_filt_model_output_temp_s1',
#                     'ewm_average_8hr_iir_filt_model_output_temp_s3',
#                     'ewm_average_8hr_iir_filt_model_output_temp_s4',
#                     'estimate_standard_deviation_8hr_iir_filt_model_output_temp_s1',
#                     'estimate_standard_deviation_8hr_iir_filt_model_output_temp_s3',
#                     'estimate_standard_deviation_8hr_iir_filt_model_output_temp_s4'
#                     ],dtype='object')
with open (r'C:\Users\lwang\OneDrive - Biolinq Inc\Gen 1\Algorithm Development\neural network glucose model\all_gModel_all_ch_3rdAttemptAllData.p','wb') as fp:
    pickle.dump(all_gModel,fp) 